In [16]:


from torch.utils.tensorboard import SummaryWriter

run_name = "run_20250515_135519"
writer = SummaryWriter(log_dir=f"../runs/CoCoOp/{run_name}")

In [14]:
writer.add_scalar("novel_classes/accuracy" , 0.7424, -1)

In [17]:
writer.add_scalar("base_classes/accuracy" , 0.7133, -1)

In [17]:
writer.close()

In [24]:

import seaborn as sns


In [25]:
from scipy import stats
import tensorboard as tb
from packaging import version

import pandas as pd
from matplotlib import pyplot as plt

In [26]:
major_ver, minor_ver, _ = version.parse(tb.__version__).release
assert major_ver >= 2 and minor_ver >= 3, \
    "This notebook requires TensorBoard 2.3 or later."
print("TensorBoard version: ", tb.__version__)

TensorBoard version:  2.19.0


In [28]:
experiment_id = f"../runs/CoCoOp/{run_name}"
experiment = tb.experimental.ExperimentFromDev(experiment_id)
df = experiment.get_scalars()
df

AttributeError: module 'tensorboard' has no attribute 'experimental'

In [23]:
from tensorboard.backend.event_processing import event_accumulator
import pandas as pd


def parse_tensorboard(path, scalars):
    """returns a dictionary of pandas dataframes for each requested scalar"""
    ea = event_accumulator.EventAccumulator(
        path,
        size_guidance={event_accumulator.SCALARS: 0},
    )
    _absorb_print = ea.Reload()
    # make sure the scalars are in the event accumulator tags
    assert all(
        s in ea.Tags()["scalars"] for s in scalars
    ), "some scalars were not found in the event accumulator"
    return {k: pd.DataFrame(ea.Scalars(k)) for k in scalars}

In [2]:
parse_tensorboard("../runs/CoCoOp/run_20250515_135519", ["base_classes/accuracy"])["base_classes/accuracy"]

,wall_time,step,value
0,1.747318e+09,8,0.73797
1,1.747323e+09,-1,0.71330
2,1.747323e+09,-1,0.71330


In [33]:
def harmonic_mean_calc(a, b):
    """returns the harmonic mean of a and b"""
    print(f"a: {a}, b: {b}")
    return 2 * (a * b) / (a + b)

In [30]:
filenames = [
    "../runs/CoCoOp/run_20250511_172826",
    "../runs/CoCoOp/run_20250515_141551",
    "../runs/CoCoOp/run_20250515_142955",
    "../runs/CoCoOp/run_20250517_161254"
]

tensorboards = {path: parse_tensorboard(path, ["base_classes/accuracy", "novel_classes/accuracy"]) for path in filenames}
tensorboards

{'../runs/CoCoOp/run_20250511_172826': {'base_classes/accuracy':       wall_time  step     value
  0  1.746985e+09    -1  0.713708
  1  1.746986e+09    10  0.813991,
  'novel_classes/accuracy':       wall_time  step     value
  0  1.746985e+09    -1  0.742383
  1  1.746986e+09    10  0.714363},
 '../runs/CoCoOp/run_20250515_141551': {'base_classes/accuracy':       wall_time  step     value
  0  1.747319e+09    -1  0.713304
  1  1.747319e+09     8  0.737970,
  'novel_classes/accuracy':       wall_time  step     value
  0  1.747319e+09    -1  0.742383
  1  1.747319e+09     8  0.737758},
 '../runs/CoCoOp/run_20250515_142955': {'base_classes/accuracy':       wall_time  step     value
  0  1.747319e+09    -1  0.713304
  1  1.747320e+09    10  0.728265,
  'novel_classes/accuracy':       wall_time  step     value
  0  1.747319e+09    -1  0.742383
  1  1.747320e+09    10  0.743743},
 '../runs/CoCoOp/run_20250517_161254': {'base_classes/accuracy':       wall_time  step     value
  0  1.747498e+

In [31]:
def step_value(df, step):
    """returns the value of the column at the step"""
    return df[df["step"] == step]["value"].values[0]

In [34]:
harmonic_means = {}
for path, tb in tensorboards.items():
    base_accs = tb["base_classes/accuracy"]
    novel_accs = tb["novel_classes/accuracy"]
    last_value_base_accs = step_value(base_accs, base_accs["step"].max())
    last_value_novel_accs = step_value(novel_accs, novel_accs["step"].max())
    harmonic_means[path] = {
        "base_acc": last_value_base_accs,
        "novel_acc": last_value_novel_accs,
        "harmonic_mean": harmonic_mean_calc(last_value_novel_accs, last_value_base_accs),
    }
    
harmonic_means
    

a: 0.7143634557723999, b: 0.813991129398346
a: 0.737758457660675, b: 0.7379700541496277
a: 0.7437431812286377, b: 0.7282652854919434
a: 0.7344940304756165, b: 0.7638495564460754


{'../runs/CoCoOp/run_20250511_172826': {'base_acc': np.float64(0.813991129398346),
  'novel_acc': np.float64(0.7143634557723999),
  'harmonic_mean': np.float64(0.7609301163579371)},
 '../runs/CoCoOp/run_20250515_141551': {'base_acc': np.float64(0.7379700541496277),
  'novel_acc': np.float64(0.737758457660675),
  'harmonic_mean': np.float64(0.7378642407353305)},
 '../runs/CoCoOp/run_20250515_142955': {'base_acc': np.float64(0.7282652854919434),
  'novel_acc': np.float64(0.7437431812286377),
  'harmonic_mean': np.float64(0.7359228597602563)},
 '../runs/CoCoOp/run_20250517_161254': {'base_acc': np.float64(0.7638495564460754),
  'novel_acc': np.float64(0.7344940304756165),
  'harmonic_mean': np.float64(0.748884226939881)}}

In [35]:
for path, stats in harmonic_means.items():
    print(f"{path}: {stats}")
    writer = SummaryWriter(log_dir=path)
    #writer.add_scalars('Final metrics', {
    #    'Harmonic Mean': harmonic_mean,
    #}, global_step=0)
    writer.add_hparams(stats, {})
    writer.close()

../runs/CoCoOp/run_20250511_172826: {'base_acc': np.float64(0.813991129398346), 'novel_acc': np.float64(0.7143634557723999), 'harmonic_mean': np.float64(0.7609301163579371)}
../runs/CoCoOp/run_20250515_141551: {'base_acc': np.float64(0.7379700541496277), 'novel_acc': np.float64(0.737758457660675), 'harmonic_mean': np.float64(0.7378642407353305)}
../runs/CoCoOp/run_20250515_142955: {'base_acc': np.float64(0.7282652854919434), 'novel_acc': np.float64(0.7437431812286377), 'harmonic_mean': np.float64(0.7359228597602563)}
../runs/CoCoOp/run_20250517_161254: {'base_acc': np.float64(0.7638495564460754), 'novel_acc': np.float64(0.7344940304756165), 'harmonic_mean': np.float64(0.748884226939881)}


In [22]:
tb = parse_tensorboard("../runs/CoCoOp/run_20250517_161254", ["Final metrics"])

AssertionError: some scalars were not found in the event accumulator

In [ ]:
import torch

state_dict = torch.load("path/to/your_prompt_learner.pth", map_location="cpu")

# Check key names
print(state_dict.keys())  # should contain something like 'ctx'

# Extract and save ctx
if "ctx" in state_dict:
    torch.save({"ctx": state_dict["ctx"]}, "path/to/your_ctx.pth")
else:
    raise KeyError("Key 'ctx' not found in the state_dict!")